In [20]:
# IMPORTS
import typing as tp
import discogs_client as DiscogsClient
from rapidfuzz import process as r_process, fuzz as r_fuzz
from rich import print as rprint
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
import json
import pathlib
import dotenv
env_file_name = '/../../../.env'
env_path = pathlib.Path(os.getcwd() + env_file_name)
if env_path.exists():
    dotenv.load_dotenv(env_path, override=True)
else:
    raise FileExistsError('env file does not exist')

SPOTIPY_CLIENT_ID = os.getenv('SPOTIFY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = os.getenv('SPOTIFY_CLIENT_SECRET')
DISCOGS_USER_TOKEN = os.getenv('DISCOGS_USER_TOKEN')

discogs_client = DiscogsClient.Client('ExampleApplication/0.1', user_token=DISCOGS_USER_TOKEN)
spotify_auth_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET)
spotify_client = spotipy.Spotify(client_credentials_manager=spotify_auth_manager)



# DISCOGS

# SPOTIFY

# YOUTUBE

Song and video are generally treated the same, but you can't add video to a playlist.

In [ ]:
# Setup
# This does not work for some reason - the device connection thingy requires identifying the device
# Much check how to avoid that.

# from ytmusicapi import YTMusic, OAuthCredentials
# YTMUSIC_REFRESH_TOKEN = os.getenv('YTMUSIC_REFRESH_TOKEN')
# YTMUSIC_ACCESS_TOKEN = os.getenv('YTMUSIC_ACCESS_TOKEN')
# YTMUSIC_CLIENT_ID = os.getenv('YTMUSIC_CLIENT_ID')
# YTMUSIC_CLIENT_SECRET = os.getenv('YTMUSIC_CLIENT_SECRET')
# with open('../../oauth.json') as file:
#     oauth = json.load(file)
# oauth['access_token'] = YTMUSIC_ACCESS_TOKEN
# oauth['refresh_token'] = YTMUSIC_REFRESH_TOKEN
# oauth_credentials = OAuthCredentials(client_id=YTMUSIC_CLIENT_ID, client_secret=YTMUSIC_CLIENT_SECRET)
# yt = YTMusic(oauth, oauth_credentials=oauth_credentials)

# Alt-authorisation
# ytmusicapi.setup()

In [17]:
# Alternative
# Use the API unauthorised

from ytmusicapi import YTMusic
youtube_client = YTMusic()


In [ ]:
filter: tp.Literal['songs', 'videos', 'albums', 'artists', 'playlists', 'community_playlists', 'featured_playlists', 'uploads']
youtube_client.search(
    'query', # str
    'filter', # filter
    'limit', # int
)


TypeError: SearchMixin.search() missing 1 required positional argument: 'query'

In [ ]:
json_path = pathlib.Path(os.getcwd() + '/../../oauth.json').resolve()
if json_path.exists():
    youtube_client = YTMusic(str(json_path), oauth_credentials=oauth_credentials)
else:
    raise FileExistsError
    
youtube_client.search('Oasis Wonderwall')

YTMusicServerError: Server returned HTTP 400: Bad Request.
Request contains an invalid argument.